In [116]:
import pandas as pd

In [117]:
df_fake_kaggle = pd.read_csv('Fake_Kaggle/train.csv')
df_fake_kaggle = df_fake_kaggle[(df_fake_kaggle['class'] == 'Fake') | (df_fake_kaggle['class'] == 'Real')]
df_fake_kaggle['class'] = df_fake_kaggle['class'].apply(lambda x : 0 if x == 'Real' else 1)
df_val_fake_kaggle = pd.DataFrame(pd.read_csv('Fake_Kaggle/test.csv').text,columns=['text'])

In [118]:
def GetFileTest(file_name):
    df = pd.read_csv(file_name)
    df = df.loc[:,['text','label']]
    df.dropna(inplace=True)
    df['label'] = df['label'].apply(lambda x : 0 if x == 1 else 1)
    return df

df_train_fake_news_kaggle = GetFileTest('Fake_News_Kaggle/train.csv')
df_val_fake_news_kaggle = pd.read_csv('Fake_News_Kaggle/test.csv').loc[:,['text']]
df_val_fake_news_kaggle.dropna(inplace=True)

In [119]:
def GetMappingLiarDataset(files_name):
    dfs = []
    for file_name in files_name:
        df_liar = pd.read_csv(file_name,delimiter='\t')
        df_liar_mapping = df_liar.iloc[:,[1,2]]
        df_liar_mapping.columns=['IsFakeNews','text']
        df_liar_mapping['label'] = df_liar_mapping.IsFakeNews.apply(lambda x: 0 if x=='false' else 1)
        df_liar_mapping.drop('IsFakeNews',axis=1,inplace=True)
        dfs.append(df_liar_mapping)
    df_test = pd.concat([dfs[1],dfs[2]],axis=0)
    return (dfs[0],df_test)
df_train_liar_dataset, df_test_liar_datasset = GetMappingLiarDataset(['Liar_Dataset/train.tsv','Liar_Dataset/test.tsv','Liar_Dataset/valid.tsv'])

In [120]:
df_fake_misinfo = pd.read_csv('Misinfo/DataSet_Misinfo_FAKE.csv')
df_fake_misinfo['label'] = df_fake_misinfo['text'].apply(lambda x:1)
df_fake_misinfo = df_fake_misinfo.loc[:,['text','label']]
df_true_misinfo = pd.read_csv('Misinfo/DataSet_Misinfo_TRUE.csv')
df_true_misinfo['label'] = df_true_misinfo['text'].apply(lambda x:0)
df_true_misinfo = df_true_misinfo.loc[:,['text','label']]
df_true_misinfo.dropna(inplace=True)
df_misinfo = pd.concat([df_fake_misinfo,df_true_misinfo])

In [121]:
df_fake = pd.read_csv('Fake_Real_News_Kaggle/Fake.csv')
df_fake = pd.DataFrame(df_fake.text,columns=['text'])
df_fake['label'] = df_fake.text.apply(lambda x : 1)
df_real = pd.read_csv('Fake_Real_News_Kaggle/True.csv')
df_real = pd.DataFrame(df_real.text)
df_real['label'] = df_real.text.apply(lambda x : 0)
df_real_news_kaggle = pd.concat([df_fake,df_real],axis=0)

In [122]:
df_welfake = pd.read_csv('Wel_Fake_Kaggle/WELFake_Dataset.csv')
df_welfake = df_welfake.loc[:,['text','label']]
df_welfake.dropna(inplace=True)

In [123]:
df_train_input = pd.read_csv('Input_Fake_News/train.csv',delimiter=';')
df_train_input = df_train_input.loc[:,['text','label']]
df_test_input = pd.read_csv('Input_Fake_News/test.csv',delimiter=';')
df_test_input = df_test_input.loc[:,['text','label']]
df_evaluation_input = pd.read_csv('Input_Fake_News/evaluation.csv',delimiter=';')
df_evaluation_input = df_evaluation_input.loc[:,['text','label']]

In [124]:
df_final_train = pd.concat([df_train_fake_news_kaggle, df_train_liar_dataset, df_test_liar_datasset, df_misinfo, df_real_news_kaggle, df_welfake, df_train_input, df_test_input])
df_final_train

,text,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,0
1,Ever get the feeling your life circles the rou...,1
2,"Why the Truth Might Get You Fired October 29, ...",0
3,Videos 15 Civilians Killed In Single US Airstr...,0
4,Print \nAn Iranian woman has been sentenced to...,0
...,...,...
8112,VATICAN CITY (Reuters) - U.S. presidential can...,1
8113,force and fanaticism wahhabism in saudi arabi...,1
8114,"ISTANBUL (Reuters) - Turkey, whose relations w...",1
8115,Here s why Clinton and Abedin should be flippi...,0


In [125]:
import numpy as np 


In [126]:
df_fake = df_final_train[df_final_train['label'] == 1]
df_true = df_final_train[df_final_train['label'] == 0]
np.random.seed(42)
indexes_fake = np.random.randint(0,df_fake.shape[0],12500)
indexes_true = np.random.randint(0,df_true.shape[0],10000)
df_fake = df_fake.iloc[indexes_fake]
df_true = df_true.iloc[indexes_true]


In [127]:
df = pd.concat([df_fake,df_true])

BaseLine = 80% (10000/12500)

In [128]:
from nltk.tokenize import word_tokenize
df['tokenize'] = df['text'].apply(lambda x: word_tokenize(x))

In [129]:
df['tokenize'] = df['tokenize'].apply(lambda tokens: [t for t in tokens if t.isalpha()])

In [130]:
from nltk.corpus import stopwords
df['tokenize'] = df['tokenize'].apply(lambda tokens: [t for t in tokens if t not in stopwords.words('english')])

KeyboardInterrupt: 

In [131]:
def Tokenize(text):
    stop_words = stopwords.words('english')
    return [token for token in word_tokenize(text) if token.isalpha() and token not in stop_words]

In [132]:
df['tokenize'] = df['text'].apply(Tokenize)

In [133]:
df['tokenized_text'] = df['tokenize'].apply(lambda tokens:' '.join(tokens).lower())

In [134]:
df_cleaned = df.loc[:,['tokenized_text','label']]
df_cleaned

,tokenized_text,label
66453,mint press news tue oct utc an inmate makes sa...,1
13013,kabul reuters islamic state claimed responsibi...,1
30750,donald trump described many sounding like comi...,1
62491,by rmuse sat oct pm he taking conspiracy theor...,1
43627,trey gowdy chairman benghazi select committee ...,1
...,...,...
33297,amsterdam reuters at least two people killed s...,0
34518,asuncion reuters paraguayan senator mario abdo...,0
36870,reuters canadian prime minister justin trudeau...,0
3598,since obamacare became law new jobs jobs,0


In [135]:
X = df_cleaned['tokenized_text']
y = df_cleaned['label']

In [136]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X_cleaned = vectorizer.fit_transform(X)
print(vectorizer.get_feature_names_out())

['aa' 'aaa' 'aaahhh' ... 'يوم' 'يوميا' '中文']


In [137]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_cleaned,y,test_size=0.2)

In [141]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
lreg = LogisticRegression()
lreg.fit(X_train, y_train)
y_pred = lreg.predict(X_test)
accuracy_score(y_test, y_pred), f1_score(y_test, y_pred)

(0.7366666666666667, 0.7733792312105565)

In [142]:
from sklearn.svm import SVC
svc = SVC()
svc.fit(X_train, y_train)
y_pred = svc.predict(X_test)
accuracy_score(y_test, y_pred), f1_score(y_test, y_pred)

(0.7373333333333333, 0.7711076684740512)

In [143]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)
accuracy_score(y_test, y_pred_knn), f1_score(y_test, y_pred_knn)

(0.5842222222222222, 0.7181804488627807)